In [1]:
from chat_GPT import Chat
import pandas as pd

In [13]:
data = pd.read_csv("../data/test/test.csv")
data

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [14]:
del data['id']
data = data[:880]

In [4]:
messages = list(data['comment_text'])

In [5]:
responses = []
chat = Chat()

for i in range(0, len(messages), 10):
    chunk = messages[i:i+10] 
    responses += chat.batch_create_chats(chunk) 


BadRequestError: Error code: 400 - {'error': {'message': "Sorry! We've encountered an issue with repetitive patterns in your prompt. Please try again with a different prompt.", 'type': 'invalid_request_error', 'param': 'prompt', 'code': 'invalid_prompt'}}

In [16]:
data.loc[:, 'gpt_classification'] = responses

data.to_csv("../data/test/test_labeled_gpt_first_880.csv", index=False)

,comment_text,gpt_classification
0,Yo bitch Ja Rule is more succesful then you'll...,1
1,== From RfC == \n\n The title is fine as it is...,0
2,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",0
3,":If you have a look back at the source, the in...",0
4,I don't anonymously edit articles at all.,0
...,...,...
875,== Update economy size == \n\n According to th...,0
876,shubaluba dingdong wang suckers!!!!!Insert non...,1
877,hi i like dicks,1
878,""" \n\n == Social conservatism == \n\n I have t...",0


In [26]:
labels = pd.read_csv("../data/test_labels/test_labels.csv")
labels

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,-1,-1,-1,-1,-1,-1


In [27]:
def fun(x):
    if x.sum() > 0:
        return 1
    elif x.sum() == 0:
        return 0
    return -1

rows = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

labels['hate'] = labels[rows].apply(lambda x: fun(x), axis=1)

for i in rows:
    del labels[i]
del labels['id']

In [31]:
sum(labels['hate'] == -1)

89186